In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline

In [2]:
df_200 = pd.read_csv("cc_test.csv")
df_1k = pd.read_csv("cc_test_1k.csv")

In [3]:
df_200.head()

,id,fkey_reference_id,company,state,gender,age,marital,premium,benefit,inflation,schedule,total_ltc,face_amount,monthly_ltc,total_ltc_80,monthly_ltc_80,timestamp
0,123440,1115,Nationwide CareMatters II,AL,Female,40,Married,60000,6 Years,3% Compound,Single Pay,307778,102593,4275,1050839.0,13538.0,9/23/2019 2:21:12 AM
1,123441,1115,Nationwide CareMatters II,AL,Female,41,Married,60000,6 Years,3% Compound,Single Pay,284222,94741,3948,942147.0,12138.0,9/23/2019 2:25:29 AM
2,123442,1115,Nationwide CareMatters II,AL,Female,42,Married,60000,6 Years,3% Compound,Single Pay,278261,92754,3865,895522.0,11537.0,9/23/2019 2:26:47 AM
3,123443,1115,Nationwide CareMatters II,AL,Female,43,Married,60000,6 Years,3% Compound,Single Pay,272546,90849,3785,851580.0,10971.0,9/23/2019 2:26:59 AM
4,123444,1115,Nationwide CareMatters II,AL,Female,44,Married,60000,6 Years,3% Compound,Single Pay,267060,89020,3709,810135.0,10437.0,9/23/2019 2:27:11 AM


In [4]:
df_1k.head()

,id,fkey_reference_id,company,state,gender,age,marital,premium,benefit,inflation,schedule,total_ltc,face_amount,monthly_ltc,total_ltc_80,monthly_ltc_80,timestamp,monthly_ltc_85,total_ltc_85
0,2229679,1704,Lincoln MoneyGuard III,WY,Male,54,None,60000,6 Years,3% Compound,10 Years,191860,64266,2472,413763,5331,12/30/2019 12:32:13 AM,6180,479675
1,2229678,1704,Lincoln MoneyGuard III,WY,Male,53,None,60000,6 Years,3% Compound,10 Years,195937,65631,2524,435232,5607,12/30/2019 12:31:59 AM,6500,504564
2,2229677,1704,Lincoln MoneyGuard III,WY,Male,52,None,60000,6 Years,3% Compound,10 Years,200195,67058,2579,458032,5901,12/30/2019 12:31:45 AM,6841,530996
3,2229676,1704,Lincoln MoneyGuard III,WY,Male,51,None,60000,6 Years,3% Compound,10 Years,203137,68043,2617,478705,6167,12/30/2019 12:31:31 AM,7149,554963
4,2229675,1704,Lincoln MoneyGuard III,WY,Male,50,None,60000,6 Years,3% Compound,10 Years,206167,69058,2656,500423,6447,12/30/2019 12:31:17 AM,7474,580140


In [5]:
# Drop empty rows
df_200 = df_200[-df_200.isnull().any(axis=1)]
df_1k = df_1k[-df_1k.isnull().any(axis=1)]

In [6]:
# Drop unnecessary columns
unnecessary = ['id', 'fkey_reference_id', 'face_amount',
               'monthly_ltc', 'total_ltc_80', 'monthly_ltc_80',
               'timestamp', 'benefit']
unnecessary1k = ['id', 'fkey_reference_id', 'face_amount',
               'monthly_ltc', 'total_ltc_80', 'monthly_ltc_80',
               'timestamp', 'benefit', 'monthly_ltc_85', 'total_ltc_85']
df_200 = df_200.drop(columns=unnecessary)
df_1k = df_1k.drop(columns=unnecessary1k)

In [7]:
df_200.loc[df_200['premium']=='$50,000', 'premium'] = '50000'
df_200.loc[df_200['premium']=='$130,000', 'premium'] = '130000'
df_200.loc[df_200['premium']=='$140,000', 'premium'] = '140000'

df_1k.loc[df_1k['premium']=='$50,000', 'premium'] = '50000'
df_1k.loc[df_1k['premium']=='$130,000', 'premium'] = '130000'
df_1k.loc[df_1k['premium']=='$140,000', 'premium'] = '140000'

/home/jayascript/.virtualenvs/datasci/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [8]:
# Encode states manually (alphabetical order)
states = df_200['state'].unique().tolist()
number_states = {'state': {k: v for k, v in zip(states,list(range(1,len(states)+1)))}}

df_200.replace(number_states, inplace=True)
df_1k.replace(number_states, inplace=True)

In [9]:
# Encode inflation manually (ranking order)
inflation_mapping = {'inflation': {'None': 0, '3% Compound': 1, '5% Simple': 2}}

df_200.replace(inflation_mapping, inplace=True)
df_1k.replace(inflation_mapping, inplace=True)

In [10]:
# Encode remaining categorical vars
categories = ['company', 'gender', 'marital', 'schedule']

df_200 = pd.get_dummies(df_200, columns=categories)
df_1k = pd.get_dummies(df_1k, columns=categories)

In [11]:
df_1k.head() # Notice that the dummy variables didn't work because there's only one company here

,state,age,premium,inflation,total_ltc,company_Lincoln MoneyGuard III,gender_Female,gender_Male,marital_Married,marital_None,schedule_10 Years,schedule_Single Pay
0,50,54,60000,1,191860,1,0,1,0,1,1,0
1,50,53,60000,1,195937,1,0,1,0,1,1,0
2,50,52,60000,1,200195,1,0,1,0,1,1,0
3,50,51,60000,1,203137,1,0,1,0,1,1,0
4,50,50,60000,1,206167,1,0,1,0,1,1,0


The test dataset isn't varied enough, so the algorithm isn't going to be nearly as accurate. We'll keep this in mind when the algorithm runs. For now, I'm just going to test it on the algo developed by the model trained on `df_200`.

In [12]:
df_200 = df_200.astype({'premium': 'int64'})

In [13]:
df_1k = df_1k.astype({'state': 'int64', 'inflation': 'int64'})

In [14]:
init_frame_200 = df_200.copy()
init_frame_1k = df_1k.copy()

In [15]:
# Train the model
X = init_frame_200.drop('total_ltc', 1)
y = init_frame_200['total_ltc']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [17]:
# Use statsmodels for variable selection
X_train = sm.add_constant(X_train)
statsmodel1 = sm.OLS(y_train, X_train).fit()
statsmodel1.summary()

/home/jayascript/.virtualenvs/datasci/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              total_ltc   R-squared:                       0.984
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                     963.0
Date:                Wed, 29 Jan 2020   Prob (F-statistic):          5.25e-132
Time:                        21:08:31   Log-Likelihood:                -1870.4
No. Observations:                 163   AIC:                             3763.
Df Residuals:                     152   BIC:                             3797.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
const                                         1.214e+05   1.27e+04      9.560      0.000    9.63e+04    1.46e+05
state                                        -4.645e-11    272.224  -1.71e-13      1.000    -537.832     537.832
age                                          -9462.4357    266.614    -35.491      0.000   -9989.184   -8935.687
premium                                          7.3700      0.303     24.330      0.000       6.771       7.968
inflation                                    -1.529e+05   1.87e+04     -8.159      0.000    -1.9e+05   -1.16e+05
company_Lincoln MoneyGuard II 2020           -4.543e+04   1.87e+04     -2.434      0.016   -8.23e+04   -8560.234
company_Lincoln MoneyGuard III                1.095e+05   2.04e+04      5.361      0.000    6.92e+04     1.5e+05
company_Nationwide CareMatters II            -2.375e+04   1.05e+04     -2.263      0.025   -4.45e+04   -3014.547
company_Pacific Life PremierCare Choice 2019  4.015e+04   1.62e+04      2.475      0.014    8094.672    7.22e+04
company_Securian Financial SecureCare UL      4.088e+04   1.71e+04      2.395      0.018    7157.817    7.46e+04
gender_Female                                 6.915e+04   1.08e+04      6.396      0.000    4.78e+04    9.05e+04
gender_Male                                   5.222e+04   5625.512      9.282      0.000    4.11e+04    6.33e+04
marital_Married                               4.685e+04   1.86e+04      2.520      0.013    1.01e+04    8.36e+04
marital_None                                  7.452e+04   8000.372      9.315      0.000    5.87e+04    9.03e+04
schedule_10 Years                            -8.926e+04   1.83e+04     -4.880      0.000   -1.25e+05   -5.31e+04
schedule_Single Pay                           2.106e+05   1.31e+04     16.096      0.000    1.85e+05    2.36e+05
==============================================================================
Omnibus:                        9.675   Durbin-Watson:                   2.174
Prob(Omnibus):                  0.008   Jarque-Bera (JB):                9.712
Skew:                          -0.535   Prob(JB):                      0.00778
Kurtosis:                       3.534   Cond. No.                     2.65e+21
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.8e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [18]:
# State is an insignificant feature, so remove it
init_frame_200 = init_frame_200.drop('state', 1)
init_frame_1k = init_frame_1k.drop('state', 1)

In [19]:
# Retrain with scikit-learn
X = init_frame_200.drop('total_ltc', 1)
y = init_frame_200['total_ltc']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [20]:
pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])

,Coefficient
age,-9462.435747
premium,5.815945
inflation,-121804.627576
company_Lincoln MoneyGuard II 2020,-91458.608490
company_Lincoln MoneyGuard III,32412.555509
company_Nationwide CareMatters II,-69785.039410
company_Pacific Life PremierCare Choice 2019,25201.815269
company_Securian Financial SecureCare UL,103629.277121
gender_Female,-7073.157438
gender_Male,7073.157438


In [21]:
# Test the algorithm
y_hat = regressor.predict(X_val)
results = pd.DataFrame({'Actual': y_val, 'Predicted': y_hat})
results

,Actual,Predicted
18,195965,164743.196628
45,211043,211043.000000
33,211043,211043.000000
37,211043,211043.000000
109,511702,503037.324786
90,384337,370563.224322
5,261790,287754.861345
124,374406,361100.788574
12,225501,221517.811113
153,314122,323595.719042


In [22]:
r2 = metrics.r2_score(y_val, y_hat)
adj_r2 = (1 - (1 - r2) * ((X_val.shape[0] - 1) / 
          (X_val.shape[0] - X_val.shape[1] - 1)))
r2, adj_r2

(0.967715456619045, 0.9503314717216077)

In [23]:
# Test the model on completely unseen data
X_test = init_frame_1k.drop('total_ltc', 1)
y_test = init_frame_1k['total_ltc']

In [24]:
# y_pred = regressor.predict(X_test)

The above code returns an error because there's a column mismatch. I'll need to populate the test datasets with the appropriate columns before I can run the regression.

In [25]:
init_frame_200.head()

,age,premium,inflation,total_ltc,company_Lincoln MoneyGuard II 2020,company_Lincoln MoneyGuard III,company_Nationwide CareMatters II,company_Pacific Life PremierCare Choice 2019,company_Securian Financial SecureCare UL,gender_Female,gender_Male,marital_Married,marital_None,schedule_10 Years,schedule_Single Pay
0,40,60000,1,307778,0,0,1,0,0,1,0,1,0,0,1
1,41,60000,1,284222,0,0,1,0,0,1,0,1,0,0,1
2,42,60000,1,278261,0,0,1,0,0,1,0,1,0,0,1
3,43,60000,1,272546,0,0,1,0,0,1,0,1,0,0,1
4,44,60000,1,267060,0,0,1,0,0,1,0,1,0,0,1


In [26]:
init_frame_1k['company_Lincoln MoneyGuard II 2020'] = 0

In [27]:
init_frame_1k['company_Nationwide CareMatters II'] = 0
init_frame_1k['company_Pacific Life PremierCare Choice 2019'] = 0
init_frame_1k['company_Securian Financial SecureCare UL'] = 0

In [28]:
init_frame_1k.shape

(1000, 15)

In [29]:
init_frame_200.shape

(204, 15)

In [30]:
# Retry the predictions
X_test = init_frame_1k.drop('total_ltc', 1)
y_test = init_frame_1k['total_ltc']

y_pred = regressor.predict(X_test)
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
results

,Actual,Predicted
0,191860,101964.815563
1,195937,111427.251311
2,200195,120889.687058
3,203137,130352.122806
4,206167,139814.558553
...,...,...
995,231478,-36630.981729
996,242654,-27168.545982
997,250725,-17706.110234
998,259352,-8243.674487


In [31]:
results[900:1000]

,Actual,Predicted
900,265124,130393.818260
901,273723,139856.254007
902,282897,149318.689755
903,292711,158781.125502
904,299641,168243.561249
...,...,...
995,231478,-36630.981729
996,242654,-27168.545982
997,250725,-17706.110234
998,259352,-8243.674487


In [32]:
r2 = metrics.r2_score(y_test, y_pred)
adj_r2 = (1 - (1 - r2) * ((X_test.shape[0] - 1) / 
          (X_test.shape[0] - X_test.shape[1] - 1)))
r2, adj_r2

(-0.9712608242256455, -0.9992787445699696)

Horrible! Let's see if we should train it on more data.

In [33]:
X = init_frame_1k.drop('total_ltc', 1)
y = init_frame_1k['total_ltc']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [35]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [36]:
pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])

,Coefficient
age,-5.965691e+03
premium,3.425055e+00
inflation,-7.275958e-12
company_Lincoln MoneyGuard III,1.091394e-11
gender_Female,-3.053407e+04
gender_Male,3.053407e+04
marital_Married,1.146455e+04
marital_None,-1.146455e+04
schedule_10 Years,-3.644118e+04
schedule_Single Pay,3.644118e+04


In [37]:
y_pred = regressor.predict(X_test)
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
r2 = metrics.r2_score(y_test, y_pred)
adj_r2 = (1 - (1 - r2) * ((X_test.shape[0] - 1) / 
          (X_test.shape[0] - X_test.shape[1] - 1)))
r2, adj_r2

(0.9319222073424326, 0.9267703743845627)

In [38]:
# Now try the highly-trained model on the small dataset
X_val = init_frame_200.drop('total_ltc', 1)
y_val = init_frame_200['total_ltc']

y_hat = regressor.predict(X_val)
r2 = metrics.r2_score(y_val, y_hat)
adj_r2 = (1 - (1 - r2) * ((X_val.shape[0] - 1) / 
          (X_val.shape[0] - X_val.shape[1] - 1)))
r2, adj_r2

(0.5649546551280669, 0.5327290740264421)

In [39]:
# Train on an even larger dataset; 24k rows
twentyk = pd.read_csv("cc_test_12k_start.csv")
additional = pd.read_csv("cc_test_12k_lincoln.csv")

In [40]:
twenty = twentyk.append(additional)

In [41]:
twenty.head()

,id,fkey_reference_id,company,state,gender,age,marital,premium,benefit,inflation,schedule,total_ltc,face_amount,monthly_ltc,total_ltc_80,monthly_ltc_80,timestamp,monthly_ltc_85,total_ltc_85
0,1769744,3215,Securian Financial SecureCare UL,SD,Female,47,Married,140000,6 Years,3% Compound,Single Pay,618357,191193,7966,1640091,21129,1/10/2020 6:50:00 AM,24495,1901357
1,1769745,3215,Securian Financial SecureCare UL,SD,Female,48,Married,140000,6 Years,3% Compound,Single Pay,627106,193898,8079,1614850,20804,1/10/2020 6:50:14 AM,24118,1872096
2,1769746,3215,Securian Financial SecureCare UL,SD,Female,55,Married,140000,6 Years,3% Compound,Single Pay,586555,181360,7557,1228117,15822,1/10/2020 6:51:51 AM,18342,1423756
3,1769747,3215,Securian Financial SecureCare UL,SD,Female,56,Married,140000,6 Years,3% Compound,Single Pay,574476,177625,7401,1167791,15045,1/10/2020 6:52:05 AM,17442,1353820
4,1769748,3215,Securian Financial SecureCare UL,SD,Female,57,Married,140000,6 Years,3% Compound,Single Pay,562396,173890,7245,1109937,14299,1/10/2020 6:52:18 AM,16577,1286750


In [42]:
### Clean dataset
# Drop empty rows
twenty = twenty[-twenty.isnull().any(axis=1)]

# Drop unnecessary columns
unnecessary = ['id', 'fkey_reference_id', 'face_amount',
               'monthly_ltc', 'total_ltc_80', 'monthly_ltc_80',
               'timestamp', 'benefit', 'monthly_ltc_85', 'total_ltc_85']
twenty = twenty.drop(columns=unnecessary)

# Clean up prices
twenty.loc[twenty['premium']=='$50,000', 'premium'] = '50000'
twenty.loc[twenty['premium']=='$130,000', 'premium'] = '130000'
twenty.loc[twenty['premium']=='$140,000', 'premium'] = '140000'

# Encode states manually (alphabetical order)
states = twenty['state'].unique().tolist()
number_states = {'state': {k: v for k, v in zip(states,list(range(1,len(states)+1)))}}
twenty.replace(number_states, inplace=True)

# Encode inflation manually (ranking order)
inflation_mapping = {'inflation': {'None': 0, '3% Compound': 1, '5% Simple': 2, '5% Compound (actually 5% Simple)': 2}}
twenty.replace(inflation_mapping, inplace=True)

# Encode remaining categorical vars
categories = ['company', 'gender', 'marital', 'schedule']
twenty = pd.get_dummies(twenty, columns=categories)

# Encode premium as integer
twenty = twenty.astype({'premium': 'int64', 'inflation': 'int64'})

# Drop state as we know it's an unnecessary column
twenty = twenty.drop('state', 1)

/home/jayascript/.virtualenvs/datasci/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [43]:
# Check dataset
twenty.head()

,age,premium,inflation,total_ltc,company_Lincoln MoneyGuard III,company_Nationwide CareMatters II,company_Pacific Life PremierCare Choice 2019,company_Securian Financial SecureCare UL,gender_Female,gender_Male,marital_Married,marital_None,schedule_10 Years,schedule_5 Years,schedule_Single Pay
0,47,140000,1,618357,0,0,0,1,1,0,1,0,0,0,1
1,48,140000,1,627106,0,0,0,1,1,0,1,0,0,0,1
2,55,140000,1,586555,0,0,0,1,1,0,1,0,0,0,1
3,56,140000,1,574476,0,0,0,1,1,0,1,0,0,0,1
4,57,140000,1,562396,0,0,0,1,1,0,1,0,0,0,1


In [44]:
twenty.dtypes

age                                             int64
premium                                         int64
inflation                                       int64
total_ltc                                       int64
company_Lincoln MoneyGuard III                  uint8
company_Nationwide CareMatters II               uint8
company_Pacific Life PremierCare Choice 2019    uint8
company_Securian Financial SecureCare UL        uint8
gender_Female                                   uint8
gender_Male                                     uint8
marital_Married                                 uint8
marital_None                                    uint8
schedule_10 Years                               uint8
schedule_5 Years                                uint8
schedule_Single Pay                             uint8
dtype: object

In [45]:
# Train the model
init_frame = twenty.copy()

X = init_frame.drop('total_ltc', 1)
y = init_frame['total_ltc']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [46]:
pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])

,Coefficient
age,-9379.837212
premium,4.116571
inflation,-165382.461900
company_Lincoln MoneyGuard III,-13840.505736
company_Nationwide CareMatters II,-22254.356144
company_Pacific Life PremierCare Choice 2019,68646.620669
company_Securian Financial SecureCare UL,-32551.758789
gender_Female,-21930.851185
gender_Male,21930.851185
marital_Married,14857.887785


In [47]:
y_pred = regressor.predict(X_test)
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
r2 = metrics.r2_score(y_test, y_pred)
adj_r2 = (1 - (1 - r2) * ((X_test.shape[0] - 1) / 
          (X_test.shape[0] - X_test.shape[1] - 1)))
r2, adj_r2

(0.8667181456271529, 0.866458445423782)

In [51]:
# Now we test again on the smaller dataset

unseen_X = df_200.drop('total_ltc', 1)
unseen_y = df_200['total_ltc']

# [Ensure the columns match](https://stackoverflow.com/questions/41335718/keep-same-dummy-variable-in-training-and-testing-data)
# Get missing columns in the training test
missing_cols = set( X_train.columns ) - set( unseen_X.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    unseen_X[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
unseen_X = unseen_X[X_train.columns]
unseen_X.head()

,age,premium,inflation,company_Lincoln MoneyGuard III,company_Nationwide CareMatters II,company_Pacific Life PremierCare Choice 2019,company_Securian Financial SecureCare UL,gender_Female,gender_Male,marital_Married,marital_None,schedule_10 Years,schedule_5 Years,schedule_Single Pay
0,40,60000,1,0,1,0,0,1,0,1,0,0,0,1
1,41,60000,1,0,1,0,0,1,0,1,0,0,0,1
2,42,60000,1,0,1,0,0,1,0,1,0,0,0,1
3,43,60000,1,0,1,0,0,1,0,1,0,0,0,1
4,44,60000,1,0,1,0,0,1,0,1,0,0,0,1


In [52]:
# Predict and evaluate performance
y_pred = regressor.predict(unseen_X)
r2 = metrics.r2_score(unseen_y, y_pred)
adj_r2 = (1 - (1 - r2) * ((unseen_X.shape[0] - 1) / 
          (unseen_X.shape[0] - unseen_X.shape[1] - 1)))
r2, adj_r2

(0.8316979045362924, 0.8192310826500919)

In [56]:
# Test again on a slightly larger dataset

unseen_X2 = init_frame_1k.drop('total_ltc', 1)
unseen_y2 = init_frame_1k['total_ltc']

missing_cols = set( X_train.columns ) - set( unseen_X2.columns )
for c in missing_cols:
    unseen_X2[c] = 0
unseen_X2 = unseen_X2[X_train.columns]

y_pred = regressor.predict(unseen_X2)
r2 = metrics.r2_score(unseen_y2, y_pred)
adj_r2 = (1 - (1 - r2) * ((unseen_X2.shape[0] - 1) / 
          (unseen_X2.shape[0] - unseen_X2.shape[1] - 1)))
r2, adj_r2

(0.7600488906079057, 0.7566384179871044)